# Вариант 93

In [2]:
from functools import lru_cache

import numpy as np


def answer(*args):
    res = ", ".join([str(i) for i in args])
    return f"Ответ: {res}\n"

In [3]:
# === Функции для задания 1


def first_transition_probability(matrix, k):
    prev = np.copy(matrix)
    for _ in range(k - 1):
        prev = matrix_power_with_skip(matrix, prev)

    return prev


def last_transition_probability(matrix, k):
    prev, res = np.copy(matrix), np.copy(matrix)
    for t in range(k - 1):
        prev = matrix_power_with_skip(matrix, prev)
        res += prev

    return res


def avg_steps(matrix):
    prev, res = np.copy(matrix), np.copy(matrix)
    for t in range(500):
        prev = matrix_power_with_skip(matrix, prev)
        res += t * prev

    return res


def first_return_probability(matrix, k):
    _matrix = np.copy(matrix)
    p_jj = np.linalg.matrix_power

    @lru_cache(maxsize=None)
    def f_jj(_k):
        return p_jj(_matrix, _k) - sum([f_jj(m) * p_jj(_matrix, _k - m) for m in range(1, _k)])

    return np.diagonal(f_jj(k))


def avg_time_return(matrix):
    _matrix = np.copy(matrix)
    p_jj = np.linalg.matrix_power
    result = []

    @lru_cache(maxsize=None)
    def f_jj(_k):
        res = p_jj(_matrix, _k) - sum([f_jj(m) * p_jj(_matrix, _k - m) for m in range(1, _k)])
        result.append(_k * np.diagonal(res))
        return res

    f_jj(500)
    return sum(result)


def last_return_probability(matrix, k):
    _matrix = np.copy(matrix)
    p_jj = np.linalg.matrix_power
    result = []

    @lru_cache(maxsize=None)
    def f_jj(_k):
        res = p_jj(_matrix, _k) - sum([f_jj(m) * p_jj(_matrix, _k - m) for m in range(1, _k)])
        result.append(np.diagonal(res))
        return res

    f_jj(k)
    return sum(result)


def steady_state_probabilities(matrix):
    matrix_ = np.copy(matrix).transpose()
    np.fill_diagonal(matrix_, np.diagonal(matrix_) - 1)
    matrix_[-1, :] = 1

    vec_b = np.zeros(len(matrix_))
    vec_b[-1] = 1
    return np.linalg.inv(matrix_).dot(vec_b)


def matrix_power_with_skip(left, right):
    range_ = range(len(left))
    return np.array([[sum(left[i, m] * right[m, j] if m != j else 0 for m in range_) for j in range_] for i in range_])


# ==============================================================================================

# === Функции для задания 2


def create_matrix(n, m, la, mu):
    size = n + m + 1
    matrix = np.zeros((size, size))
    np.fill_diagonal(matrix[:, 1:], la)
    np.fill_diagonal(matrix[1:, :], [*[i * mu for i in range(1, m)], *[m * mu for _ in range(n + 1)]])
    return matrix


def steady_state_probability(matrix):
    size = len(matrix)
    matrix_ = np.copy(matrix).transpose()
    np.fill_diagonal(matrix_, [-sum(matrix_[:, i]) for i in range(size)])
    matrix_[-1, :] = 1

    vec_b = np.zeros(size)
    vec_b[-1] = 1
    return np.linalg.inv(matrix_).dot(vec_b)


def failure_probability(s_state):
    return s_state[-1]


def relative_and_absolute_service_intensity(s_state, la):
    relative = 1 - s_state[-1]
    return relative, relative * la


def average_queue_length(s_state, m, n):
    return sum((i * s_state[m + i]) for i in range(1, n + 1))


def average_queue_time(s_state, m, n, mu):
    return sum(((i + 1) / (m * mu) * s_state[m + i]) for i in range(n))


def average_number_busy_channels(s_state, m, n):
    return (sum((i * s_state[i]) for i in range(1, m + 1)) +
            sum((m * s_state[i]) for i in range(m + 1, m + n + 1)))


def skip_queue_probability(s_state, m):
    return sum(s_state[:m])


def average_downtime(matrix):
    return 1 / np.sum(matrix, axis=1)

# Задание 1

In [4]:
transition_matrix = np.array([
    [0.2, 0, 0, 0, 0, 0.43, 0.37, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0.36, 0.64, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0.08, 0.12, 0.53, 0, 0.27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0.53, 0, 0.47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0.21, 0.05, 0.28, 0, 0.11, 0.1, 0, 0.25, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0.26, 0.4, 0, 0, 0.13, 0, 0.2, 0, 0.01, 0, 0, 0, 0, 0, 0, 0],
    [0, 0.33, 0.3, 0, 0, 0, 0.14, 0, 0.23, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0.38, 0.12, 0.08, 0.28, 0, 0, 0.14, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0.32, 0, 0, 0, 0.08, 0, 0.16, 0.44, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0.46, 0, 0, 0.01, 0.06, 0, 0, 0.47, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0.01, 0.08, 0.19, 0, 0, 0.11, 0.24, 0.18, 0.19],
    [0, 0, 0, 0, 0, 0, 0, 0, 0.47, 0, 0.08, 0.06, 0, 0, 0.01, 0.38, 0],
    [0, 0, 0, 0, 0, 0, 0, 0.18, 0.05, 0, 0, 0.16, 0.08, 0.28, 0.04, 0, 0.21],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.36, 0.14, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.81, 0.19, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0, 0.05, 0, 0.09, 0.48, 0.18],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.34, 0.45, 0.05, 0, 0, 0.11, 0.05],
])

print("Матрица переходов:")
print(transition_matrix.sum(axis=1))
print(transition_matrix, "\n")

Матрица переходов:
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[0.2  0.   0.   0.   0.   0.43 0.37 0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.36 0.64 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.08 0.12 0.53 0.   0.27 0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.   0.   0.53 0.   0.47 0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.21 0.05 0.28 0.   0.11 0.1  0.   0.25 0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.26 0.4  0.   0.   0.13 0.   0.2  0.   0.01 0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.33 0.3  0.   0.   0.   0.14 0.   0.23 0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.38 0.12 0.08 0.28 0.   0.   0.14 0.   0.
  0.   0.   0.  ]
 [0.   0.   0.   0.   0.32 0.   0.   0.   0.08 0.   0.16 0.44 0.   0.
  0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.46 0.   0.   0.01 0.06 0.   0.   0.47 0.
  0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.01 0

In [5]:
k, i, j = 8, 14, 9
transition = np.linalg.matrix_power(transition_matrix, k)
print(f"1. вероятность того, что за {k} шагов система перейдет из состояния {i} в состояние {j}:")
print(answer(transition[i - 1, j - 1]))

1. вероятность того, что за 8 шагов система перейдет из состояния 14 в состояние 9:
Ответ: 0.08597085978784999



In [7]:
k = 10
a_0 = np.array([0.06, 0.06, 0.09, 0.06, 0.08, 0.04, 0.01, 0.1, 0.07, 0.08, 0.01, 0.04, 0.1, 0.1, 0.05, 0.04, 0.01])
state = a_0.dot(np.linalg.matrix_power(transition_matrix, k))
print(f"2. вероятности состояний системы спустя {k} шагов, если в начальный "
      f"момент вероятность состояний были следующими A={a_0}:")
print(answer(state))

2. вероятности состояний системы спустя 10 шагов, если в начальный момент вероятность состояний были следующими A=[0.06 0.06 0.09 0.06 0.08 0.04 0.01 0.1  0.07 0.08 0.01 0.04 0.1  0.1
 0.05 0.04 0.01]:
Ответ: [0.06488613 0.13218863 0.07187925 0.08101751 0.02272939 0.1184391
 0.03232777 0.03378946 0.06318813 0.00604779 0.05590455 0.08525054
 0.00963479 0.06418773 0.03933339 0.08885996 0.03033588]



In [8]:
k, i, j = 8, 6, 15
first_transition = first_transition_probability(transition_matrix, k)
print(f"3. вероятность первого перехода за {k} шагов из состояния {i} в состояние {j}:")
print(answer(first_transition[i - 1, j - 1]))

3. вероятность первого перехода за 8 шагов из состояния 6 в состояние 15:
Ответ: 0.008560630642178001



In [9]:
k, i, j = 10, 10, 8
last_transition = last_transition_probability(transition_matrix, k)
print(f"4. вероятность перехода из состояния {i} в состояние {j} не позднее чем за {k} шагов:")
print(answer(last_transition[i - 1, j - 1]))

4. вероятность перехода из состояния 10 в состояние 8 не позднее чем за 10 шагов:
Ответ: 0.35324191100135643



In [10]:
i, j = 8, 10
avg_steps_number = avg_steps(transition_matrix)
print(f"5. среднее количество шагов для перехода из состояния {i} в состояние {j}:")
print(answer(avg_steps_number[i - 1, j - 1]))

5. среднее количество шагов для перехода из состояния 8 в состояние 10:
Ответ: 138.70862871717543



In [11]:
k, i = 9, 9
first_return = first_return_probability(transition_matrix, k)
print(f"6. вероятность первого возвращения в состояние {i} за {k} шагов:")
print(answer(first_return[i - 1]))

6. вероятность первого возвращения в состояние 9 за 9 шагов:
Ответ: 0.02994476964339407



In [12]:
k, i = 9, 14
last_return = last_return_probability(transition_matrix, k)
print(f"7. вероятность возвращения в состояние {i} не позднее чем за {k} шагов")
print(answer(last_return[i - 1]))

7. вероятность возвращения в состояние 14 не позднее чем за 9 шагов
Ответ: 0.7038904138971471



In [13]:
i = 5
avg_time_to_return = avg_time_return(transition_matrix)
print(f"8. среднее время возвращения в состояние {i}")
print(answer(avg_time_to_return[i - 1]))

8. среднее время возвращения в состояние 5
Ответ: 45.45008726953896



In [14]:
steady_state = steady_state_probabilities(transition_matrix)
print("9. установившиеся вероятности:")
print(answer(steady_state))

9. установившиеся вероятности:
Ответ: [0.06944404 0.1414864  0.07538469 0.08500827 0.02200062 0.1243994
 0.03472865 0.03476956 0.06118922 0.0057528  0.0520455  0.07988592
 0.0089331  0.05856305 0.03611615 0.08231274 0.02797988]



## Задание 2

In [15]:
la = 18
m = 6
mu = 3
n = 19
initial_matrix = create_matrix(n, m, la, mu)
initial_matrix

array([[ 0., 18.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 3.,  0., 18.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  6.,  0., 18.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  9.,  0., 18.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 12.,  0., 18.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., 15.,  0., 18.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 18.,  0., 18.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [16]:
steady_state = steady_state_probability(initial_matrix)
print("a) Составьте граф марковского процесса, запишите систему уравнений Колмогорова и "
      "найдите установившиеся вероятности состояний:")
print(answer(steady_state))

a) Составьте граф марковского процесса, запишите систему уравнений Колмогорова и найдите установившиеся вероятности состояний:
Ответ: [0.0006776  0.00406559 0.01219677 0.02439355 0.03659032 0.04390839
 0.04390839 0.04390839 0.04390839 0.04390839 0.04390839 0.04390839
 0.04390839 0.04390839 0.04390839 0.04390839 0.04390839 0.04390839
 0.04390839 0.04390839 0.04390839 0.04390839 0.04390839 0.04390839
 0.04390839 0.04390839]



In [17]:
fail_probability = failure_probability(steady_state)
print("b) Найдите вероятность отказа в обслуживании:")
print(answer(fail_probability))

b) Найдите вероятность отказа в обслуживании:
Ответ: 0.043908388670551586



In [18]:
relative, absolute = relative_and_absolute_service_intensity(steady_state, la)
print("c) Найдите относительную и абсолютную интенсивность обслуживания:")
print(answer(relative, absolute))

c) Найдите относительную и абсолютную интенсивность обслуживания:
Ответ: 0.9560916113294484, 17.20964900393007



In [19]:
avg_queue_len = average_queue_length(steady_state, m, n)
print("d) Найдите среднюю длину в очереди:")
print(answer(avg_queue_len))

d) Найдите среднюю длину в очереди:
Ответ: 8.342593847404798



In [20]:
avg_queue_time = average_queue_time(steady_state, m, n, mu)
print("e) Найдите среднее время в очереди:")
print(answer(avg_queue_time))

e) Найдите среднее время в очереди:
Ответ: 0.46347743596693325



In [21]:
avg_number_channels = average_number_busy_channels(steady_state, m, n)
print("f) Найдите среднее число занятых каналов:")
print(answer(avg_number_channels))

f) Найдите среднее число занятых каналов:
Ответ: 5.736549667976691



In [22]:
skip_probability = skip_queue_probability(steady_state, m)
print("g) Найдите вероятность того, что поступающая заявка не будет ждать в очереди:")
print(answer(skip_probability))

g) Найдите вероятность того, что поступающая заявка не будет ждать в очереди:
Ответ: 0.1218322265889687



In [23]:
avg_downtime = average_downtime(initial_matrix)
print("h) Найти среднее время простоя системы массового обслуживания:")
print(answer(avg_downtime[0]))

h) Найти среднее время простоя системы массового обслуживания:
Ответ: 0.05555555555555555

